# Predicting T-shirt size using the ANSUR II dataset
We will here try to predict a persons t-shirt size given the weight and height of the person. We will use the ANSUR II dataset which contains a lot of information about the physical attributes of a large number of people. 

We will first try to map the persons in the dataset to a t-shirt size. It is hard to find a concise size chart for t-shirt so we will create our own, initial chart, based on these assumptions:

We will only look at two measurements, Shoulder Width and Chest Circumference.

Our first problem is that Shoulder Width is not one of the measurements taken in the dataset. But we have Biacromial Breadth which is the distance between the two acromion processes. We will assume that this is the same as Shoulder Width.

We will then have these initial rules:

| Size | Percentile |
|------|------------|
| XS   | 0-5        |
| S    | 5-25       |
| M    | 25-50      |
| L    | 50-75      |
| XL   | 75-90      |
| XXL  | 90-97      |
| XXXL | 97-100     |

In [28]:
import pandas as pd
female = pd.read_csv('./Data/female.csv' )
male = pd.read_csv('./Data/male.csv' )
print(f'The female shirts size data {female.shape}')
print(f'The male shirts size data {male.shape}')

The female shirts size data (1986, 108)
The male shirts size data (4082, 108)


In [30]:
def compute_percentile_ranges(column):
    # Define percentile ranges
    ranges = [(0, 5), (5, 25), (25, 50), (50, 75), (75, 90), (90, 97), (97, 100)]

    percentiles = {(low, high): (column.quantile(low/100), column.quantile(high/100)) for low, high in ranges}

    counts = {}

    for r, (low, high) in percentiles.items():
        counts[r] = ((column >= low) & (column < high)).sum()
    
    return counts

print(compute_percentile_ranges(female['chestcircumference']))
print(compute_percentile_ranges(female['biacromialbreadth']))

print(compute_percentile_ranges(male['chestcircumference']))
print(compute_percentile_ranges(male['biacromialbreadth']))



{(0, 5): np.int64(1), (5, 25): np.int64(1), (25, 50): np.int64(1), (50, 75): np.int64(1), (75, 90): np.int64(1), (90, 97): np.int64(0), (97, 100): np.int64(0)}
{(0, 5): np.int64(1), (5, 25): np.int64(1), (25, 50): np.int64(1), (50, 75): np.int64(1), (75, 90): np.int64(1), (90, 97): np.int64(0), (97, 100): np.int64(0)}
{(0, 5): np.int64(199), (5, 25): np.int64(810), (25, 50): np.int64(1025), (50, 75): np.int64(1012), (75, 90): np.int64(616), (90, 97): np.int64(295), (97, 100): np.int64(124)}
{(0, 5): np.int64(191), (5, 25): np.int64(787), (25, 50): np.int64(989), (50, 75): np.int64(1079), (75, 90): np.int64(610), (90, 97): np.int64(303), (97, 100): np.int64(122)}


In [33]:
def compute_size_percentile_mesurments(data, chest_column, shoulder_column):
    sizes = ['XS', 'S', 'M', 'L', 'XL', '2XL', '3XL']
    ranges = [0, 5, 25, 50, 75, 90, 97]

    # Compute the values for each percentile for chest and shoulder
    chest_percentiles = {p: data[chest_column].quantile(p/100) for p in ranges}
    shoulder_percentiles = {p: data[shoulder_column].quantile(p/100) for p in ranges}

    # Map the t-shirt sizes to the corresponding chest and shoulder measurments
    size_mappings = {}
    for i, size in enumerate(sizes):
        size_mappings[size] = {
            'Chest': int(chest_percentiles[ranges[i]]),
            'Shoulder': int(shoulder_percentiles[ranges[i]])
        }
    
    return size_mappings
    


data1=print(compute_size_percentile_mesurments(female, 'chestcircumference', 'biacromialbreadth'))
data2=print(compute_size_percentile_mesurments(male, 'chestcircumference', 'biacromialbreadth'))

 

{'XS': {'Chest': 839, 'Shoulder': 356}, 'S': {'Chest': 847, 'Shoulder': 360}, 'M': {'Chest': 886, 'Shoulder': 372}, 'L': {'Chest': 965, 'Shoulder': 375}, 'XL': {'Chest': 1025, 'Shoulder': 382}, '2XL': {'Chest': 1060, 'Shoulder': 390}, '3XL': {'Chest': 1080, 'Shoulder': 395}}
{'XS': {'Chest': 774, 'Shoulder': 337}, 'S': {'Chest': 922, 'Shoulder': 384}, 'M': {'Chest': 996, 'Shoulder': 403}, 'L': {'Chest': 1056, 'Shoulder': 415}, 'XL': {'Chest': 1117, 'Shoulder': 428}, '2XL': {'Chest': 1172, 'Shoulder': 441}, '3XL': {'Chest': 1233, 'Shoulder': 452}}
